# Almacenamiento Licitaciones BO

# 📦 Estructura del Proyecto

```bash
📁 TP_Final  # Carpeta raíz del proyecto
│
├── 📂 data  # Almacenamiento en Delta Lake
│   ├── 📂 bronze  
│   │   ├── 📂 boletines             # DT de boletines
│   │   ├── 📂 normas                # DT de normas, particionado por poder que emitió
│   │   ├── 📂 organismos_emisores  # DT con extracción FULL de organismos emisores
│   │   ├── 📂 reparticiones        # DT con extracción FULL de reparticiones
│   │   └── 📂 bac_anual            # DT con compras OCID y empresas
│   └── 📂 silver
│       ├── 📂 boletines
│       ├── 📂 licitaciones #DT de licitaciones enriquecido
│       ├── 📂 organismos_emisores #DT de organismos_emisores enriquecido
│       └── 📂 empresas # DT con suppliers inscriptos del GCABA
├── 📂 scripts  # Scripts de procesamiento
│   ├── 📄 boletin_oficial_api.py       # 📌 Extracción y parseo desde la API
│   ├── 📄 transformaciones.py          # 📌 Limpieza y normalización de boletines y normas
│   ├── 📄 transformaciones_licitaciones.py  # 📌 Procesamiento, extracción y enriquecimiento de licitaciones
│   ├── 📄 transformaciones_empresas.py      # 📌 Generación de padrón de empresas y enriquecimiento
│   ├── 📄 storage.py                   # 📌 Utilidades para manejo de Delta Lake (upsert, overwrite)
│   ├── 📄 utils_logs.py                   # 📌 Utilidades para manejo de errores en descarga de boletines y/o normas y logs
├── 📂 logs  # logs de extracción y descarga
│
├── 📄 almacenamiento.ipynb  # 📌 Script principal para carga en Bronze (boletines, normas, organismos)
├── 📄 procesamiento.ipynb   # 📌 Script principal de procesamiento y carga a Silver
├── 📄 requirements.txt   # 📌 Librerías necesarias (deltalake, pandas, requests, etc.)
├── 📄 README.md          # 📌 Documentación
```




# Importaciones y configuración

In [1]:
import os
import pandas as pd
import re
from datetime import datetime
from deltalake import DeltaTable,  write_deltalake
from scripts.boletin_oficial_api import BoletinOficialAPI
from scripts.storage import upsert_data
from scripts.transformaciones import limpiar_dataframe
from datetime import datetime, timedelta
from scripts.utils_logs import detectar_fechas_cubiertas_boletines, gestionar_logs_de_errores_boletines, guardar_log_errores_normas, gestionar_log_errores_normas

# Extracción Incremental

In [2]:
# Rutas a las tablas de Delta Lake
BRONZE_BOLETINES = "data/bronze/boletin_api/boletines"
BRONZE_NORMAS = "data/bronze/boletin_api/normas"
BRONZE_LICITACIONES = "data/bronze/boletin_api/licitaciones"

In [3]:
FECHA_INICIO = "10-03-2025"
FORZAR_INICIO_MANUAL = True  # ← ⚠️ Activar para ignorar lo último que haya en Bronze y forzar inicio desde fecha

# 🔎 Buscar boletines ya guardados
try:
    dt = DeltaTable(BRONZE_BOLETINES)
    df_bronze = dt.to_pandas()

    # 🔢 Números de boletines ya guardados
    boletines_existentes = set(df_bronze['numero'].dropna().astype(int).unique())

    # 📅 Fechas ya cubiertas por boletines
    fechas_cubiertas = detectar_fechas_cubiertas_boletines(df_bronze)

    # 🧹 Limpiar errores pasados ya resueltos
    gestionar_logs_de_errores_boletines(fechas_cubiertas)

    # 🕓 Determinar fecha de inicio
    if FORZAR_INICIO_MANUAL:
        fecha_inicio_actualizada = datetime.strptime(FECHA_INICIO, "%d-%m-%Y")
        print(f"📌 Usando fecha forzada: {fecha_inicio_actualizada.date()}")
    else:
        ultima_fecha = pd.to_datetime(df_bronze['fecha_publicacion'], errors='coerce').max()
        fecha_inicio_actualizada = ultima_fecha + timedelta(days=1)
        print(f"📌 Último boletín registrado en Bronze: {ultima_fecha.date()}")

except Exception:
    print("⚠️ No se encontró tabla o no se pudo leer.")
    boletines_existentes = set()
    fechas_cubiertas = set()
    fecha_inicio_actualizada = datetime.strptime(FECHA_INICIO, "%d-%m-%Y")

# 📥 Descargar boletines nuevos
fecha_str = fecha_inicio_actualizada.strftime("%d-%m-%Y")
boletines_nuevos = BoletinOficialAPI.obtener_boletines_desde_fecha(
    fecha_str, boletines_existentes, fechas_cubiertas
)


📌 Usando fecha forzada: 2025-03-10
📥 Consultando API con fecha: 10-03-2025


KeyboardInterrupt: 

# PArseo de boletines y normas

In [4]:
if not boletines_nuevos:
    print("No se encontraron boletines nuevos para procesar.")
else:
    # 📌 Se convierten los nodos de boletines y normas a DataFrames
    df_boletines = BoletinOficialAPI.parsear_boletines(boletines_nuevos)
    df_normas = BoletinOficialAPI.parsear_normas(boletines_nuevos)


In [5]:
# 🔄 Transformaciones antes de guardar
df_boletines = limpiar_dataframe(df_boletines, key_cols=["numero"])
df_normas = limpiar_dataframe(df_normas, key_cols=["id_norma"])

In [12]:
df_normas['subsecciones'].value_counts()

subsecciones
Poder Ejecutivo         528
Licitaciones            100
Comunicados y Avisos     63
Edictos Oficiales        29
Organos de Control       12
Poder Judicial           10
Fuera de Nivel            6
Trimestrales              4
Edictos Particulares      2
Name: count, dtype: int64

## Descarga de licitaciones en formato pdf y extracción de texto

In [15]:
# Procesar licitaciones
df_licitaciones = BoletinOficialAPI.procesar_licitaciones(df_normas)

# 1️⃣ Verificar si errores viejos ya se resolvieron (antes de guardar los nuevos)
gestionar_log_errores_normas(df_licitaciones)

# 2️⃣ Guardar errores de esta corrida
guardar_log_errores_normas(BoletinOficialAPI.errores_pdf_normas)

# 3️⃣ Limpiar DataFrame
df_licitaciones = limpiar_dataframe(df_licitaciones, key_cols=["id_norma"])


📑 Procesando PDF 1/100: id_norma 884700
✅ PDF ya descargado: data\bronze\boletin_api\licitaciones_pdf\884700.pdf
📑 Procesando PDF 2/100: id_norma 885047
✅ PDF ya descargado: data\bronze\boletin_api\licitaciones_pdf\885047.pdf
📑 Procesando PDF 3/100: id_norma 885065
✅ PDF ya descargado: data\bronze\boletin_api\licitaciones_pdf\885065.pdf
📑 Procesando PDF 4/100: id_norma 884909
✅ PDF ya descargado: data\bronze\boletin_api\licitaciones_pdf\884909.pdf
📑 Procesando PDF 5/100: id_norma 885104
✅ PDF ya descargado: data\bronze\boletin_api\licitaciones_pdf\885104.pdf
📑 Procesando PDF 6/100: id_norma 884758
✅ PDF ya descargado: data\bronze\boletin_api\licitaciones_pdf\884758.pdf
📑 Procesando PDF 7/100: id_norma 884755
✅ PDF ya descargado: data\bronze\boletin_api\licitaciones_pdf\884755.pdf
📑 Procesando PDF 8/100: id_norma 885069
✅ PDF ya descargado: data\bronze\boletin_api\licitaciones_pdf\885069.pdf
📑 Procesando PDF 9/100: id_norma 885063
✅ PDF ya descargado: data\bronze\boletin_api\licitacione

# Carga de datos

In [17]:
# 📌 Upsert en bronze mediante funcion definida en storage.py usando Delta Lake

print("🔼 Guardando boletines...")
upsert_data(df_boletines, 
            BRONZE_BOLETINES,
            key_col="numero",
            partition_col="anio")
print("✅ Boletines guardados en Delta Lake.")


print("🔼 Guardando normas...")
upsert_data(df_normas,
            BRONZE_NORMAS,
            key_col="id_norma",
            partition_col="tipo_norma")
print("✅ Normas guardadas en Delta Lake.")

print("🔼 Guardando licitaciones...")
upsert_data(df_licitaciones,
            BRONZE_LICITACIONES,
            key_col="id_norma",
            partition_col="fecha_publicacion")
print("✅ Licitaciones guardadas en Delta Lake.")

🔼 Guardando boletines...
📊 Tipo de 'numero' después del saneo: Int64
🔢 Registros con clave no nula: 2
📂 No existe DeltaTable en data/bronze/boletin_api/boletines o no se pudo hacer MERGE. Creando nueva tabla...
✅ Datos insertados en data/bronze/boletin_api/boletines. Filas nuevas: 2
✅ Boletines guardados en Delta Lake.
🔼 Guardando normas...
📊 Tipo de 'id_norma' después del saneo: Int64
🔢 Registros con clave no nula: 754
📂 No existe DeltaTable en data/bronze/boletin_api/normas o no se pudo hacer MERGE. Creando nueva tabla...
✅ Datos insertados en data/bronze/boletin_api/normas. Filas nuevas: 754
✅ Normas guardadas en Delta Lake.
🔼 Guardando licitaciones...
📊 Tipo de 'id_norma' después del saneo: Int64
🔢 Registros con clave no nula: 100
📂 No existe DeltaTable en data/bronze/boletin_api/licitaciones o no se pudo hacer MERGE. Creando nueva tabla...
✅ Datos insertados en data/bronze/boletin_api/licitaciones. Filas nuevas: 100
✅ Licitaciones guardadas en Delta Lake.


In [18]:

# 📊 Mostrar DeltaTable después de la inserción
dt = DeltaTable(BRONZE_BOLETINES)
df_actualizado = dt.to_pandas(columns=["numero", "fecha_publicacion"])
print("📊 DeltaTable DESPUÉS de la inserción:")
print(df_actualizado)

print("✅ Proceso completado.")

📊 DeltaTable DESPUÉS de la inserción:
   numero fecha_publicacion
0    7100        2025-04-15
1    7099        2025-04-14
✅ Proceso completado.


# Extracción full (Organismo, Reparticiones y BAC Compras)

In [ ]:
# Rutas de almacenamiento en BRONZE
FULL_ORGANISMOS = "data/bronze/boletin_api/organismos_emisores"
FULL_REPARTICIONES = "data/bronze/boletin_api/reparticiones"

## Extracción full de datos maestros de reparticiones y organismos emisores

Observación: no se identifica una clave directa para realizar el cruce entre la tabla de reparticiones y la de normas. Se evaluará en una proxima iteración la posibilidad de realizar un mapeo utilizando coincidencias de cadena de texto

In [ ]:

print("📥 Extrayendo organismos emisores...")
data_organismos = BoletinOficialAPI.obtener_organismos_emisores()
df_organismos = BoletinOficialAPI.parsear_organismos_emisores(data_organismos)

print("📥 Extrayendo reparticiones...")
data_reparticiones = BoletinOficialAPI.obtener_reparticiones()
df_reparticiones = BoletinOficialAPI.parsear_reparticiones(data_reparticiones)

# Agregar columna de fecha de extracción
fecha_actual = datetime.today().strftime("%Y-%m-%d")
df_organismos["fecha_extraccion"] = fecha_actual
df_reparticiones["fecha_extraccion"] = fecha_actual


📥 Extrayendo organismos emisores...
📥 Extrayendo reparticiones...


In [ ]:

print("Organismos emisores extraídos:", df_organismos.shape)
print("Reparticiones extraídas:", df_reparticiones.shape)

Organismos emisores extraídos: (1259, 3)
Reparticiones extraídas: (196, 4)


## Guardado en Delta Lake con sobreescritura

In [ ]:

print("💾 Guardando datos en Delta Lake...")

write_deltalake(
    FULL_ORGANISMOS,
      df_organismos,
        mode="overwrite"
)
write_deltalake(
    FULL_REPARTICIONES,
      df_reparticiones,
        mode="overwrite"
)

print("✅ Extracción FULL completada y guardada en Delta Lake.")


💾 Guardando datos en Delta Lake...
✅ Extracción FULL completada y guardada en Delta Lake.


# Extracción y almacenamiento de Dataset de BAC Compras (empresas licitantes)

Se guarda el dataset de datos abiertos sobre licitaciones en formato OCID para luego extraer las empresas en la etapa silver

In [ ]:
df_ocid_raw = pd.read_csv('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-economia-y-finanzas/buenos-aires-compras/bac_anual.csv')

C:\Users\untal\AppData\Local\Temp\ipykernel_130364\730753928.py:1: DtypeWarning: Columns (23,109,110,111) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ocid_raw = pd.read_csv('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-economia-y-finanzas/buenos-aires-compras/bac_anual.csv')


In [ ]:
# 2. Definir ruta para Bronze
BRONZE_OCID_FULL = "data/bronze/bac_anual/full_bac_compras_anual"

# 3. Guardar en Delta Lake (modo OVERWRITE para carga limpia)
write_deltalake(
    BRONZE_OCID_FULL,
    df_ocid_raw,
    mode="overwrite"
)

print("✅ Archivo OCDS completo guardado en Delta Lake (bronze/bac_anual/full_ocid)")

✅ Archivo OCDS completo guardado en Delta Lake (bronze/bac_anual/full_ocid)
